In [1]:
import pandas as pd
import numpy as np

In [2]:
df1 = pd.read_csv("H:\\Datatsets\\medium-master\\medium-master\\items-recommender\\data\\trx_data.csv")
df1.head(5)

,customerId,products
0,0,20
1,1,2|2|23|68|68|111|29|86|107|152
2,2,111|107|29|11|11|11|33|23
3,3,164|227
4,5,2|2


In [3]:
df2 = pd.read_csv("H:\\Datatsets\\medium-master\\medium-master\\items-recommender\\data\\recommend_1.csv")
df2.head(5)

,customerId
0,1553
1,20400
2,19750
3,6334
4,27773


In [4]:
df =df1.set_index('customerId')

In [5]:
df1.head(5)

,customerId,products
0,0,20
1,1,2|2|23|68|68|111|29|86|107|152
2,2,111|107|29|11|11|11|33|23
3,3,164|227
4,5,2|2


In [6]:
df1['products'] = df1['products'].apply(lambda s: [int(i) for i in s.split('|')])
df1.head(2).set_index('customerId')['products'].apply(pd.Series).reset_index()

,customerId,0,1,2,3,4,5,6,7,8,9
0,0,20.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2.0,2.0,23.0,68.0,68.0,111.0,29.0,86.0,107.0,152.0


In [7]:
data = pd.melt(df1.set_index('customerId')['products'].apply(pd.Series).reset_index(), 
             id_vars=['customerId'],
             value_name='products')

In [8]:
data.head(5)
data = data.dropna().drop(['variable'], axis=1)

In [9]:
data = data.groupby(['customerId', 'products']).agg({'products': 'count'})
data.head(5)

products
customerId products          
0          1.0              2
           13.0             1
           19.0             3
           20.0             1
           31.0             2

In [10]:
data = data.rename(columns={'products': 'purchase_count'}).reset_index() 
data.head(5)

,customerId,products,purchase_count
0,0,1.0,2
1,0,13.0,1
2,0,19.0,3
3,0,20.0,1
4,0,31.0,2


In [11]:
data = data.rename(columns={'products': 'productId'})
data.head(5)

,customerId,productId,purchase_count
0,0,1.0,2
1,0,13.0,1
2,0,19.0,3
3,0,20.0,1
4,0,31.0,2


In [12]:
data['productId'] = data['productId'].astype(np.int64)

In [13]:
data.shape

(133585, 3)

Create Dummy

In [14]:
def dummy_data(d):
    dummy_d = data.copy()
    dummy_d['bought_item_dummy'] = 1 #if prodcut bought by user, return value = 1
    return dummy_d

In [15]:
df3 = dummy_data(data)
df3.head(5)

,customerId,productId,purchase_count,bought_item_dummy
0,0,1,2,1
1,0,13,1,1
2,0,19,3,1
3,0,20,1,1
4,0,31,2,1


Normalize

In [16]:
df_matrix = pd.pivot_table(data, values='purchase_count', index='customerId', columns='productId')
df_matrix.head()

productId,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
customerId,,,,,,,,,,,,,,,,,,,,,
0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
df_matrix.shape

(24429, 300)

In [19]:
df_matrix_norm = (df_matrix-df_matrix.min())/(df_matrix.max()-df_matrix.min())
print(df_matrix_norm.shape)
df_matrix_norm.head()

(24429, 300)


productId,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
customerId,,,,,,,,,,,,,,,,,,,,,
0,NaN,0.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,0.166667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
# create a table for input to the modeling
d = df_matrix_norm.reset_index()
d.index.names = ['scaled_purchase_freq']
data_norm = pd.melt(d, id_vars=['customerId'], value_name='scaled_purchase_freq').dropna()
print(data_norm.shape)
data_norm.head()

(133585, 3)


,customerId,productId,scaled_purchase_freq
9,9,0,0.133333
25,25,0,0.133333
32,33,0,0.133333
35,36,0,0.133333
43,44,0,0.133333
